<a href="https://colab.research.google.com/github/cedalexandre/Few-Shot-Patch-Based-Training/blob/master/DEAMS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###### INITIALISATION #####

resume = False

import subprocess
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

try: 
  !pip3 install googletrans==3.1.0a0
  from googletrans import Translator, constants
  translator = Translator()
except: pass
!pip install ftfy==5.8

!apt-get -qq install ffmpeg
from google.colab import drive
drive.mount('/G', force_remount=True)
gdir = !ls /G/
gdir = '/G/%s/' % str(gdir[0])
%cd $gdir
work_dir = 'illustra'
work_dir = gdir + work_dir + '/'
import os
os.makedirs(work_dir, exist_ok=True)
%cd $work_dir

import os
import io
import time
import math
# from math import exp
import random
import imageio
import numpy as np
import PIL
# from skimage import exposure
from base64 import b64encode
import shutil
import moviepy, moviepy.editor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable

from IPython.display import HTML, Image, display, clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import ipywidgets as ipy
# import glob
from google.colab import output, files

import warnings
warnings.filterwarnings("ignore")

!pip install git+https://github.com/openai/CLIP.git
import clip
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
!pip install git+https://github.com/Po-Hsun-Su/pytorch-ssim
import pytorch_ssim as ssim

%cd /content
!rm -rf aphantasia
#!git clone https://github.com/eps696/aphantasia
! git clone https://github.com/cedalexandre/aphantasia
%cd aphantasia/
from clip_fft import to_valid_rgb, fft_image, slice_imgs, checkout
from utils import pad_up_to, basename, file_list, img_list, img_read
from progress_bar import ProgressIPy as ProgressBar

clear_output()

if resume:
  resumed = files.upload()
  params_pt = list(resumed.values())[0]
  params_pt = torch.load(io.BytesIO(params_pt))

def read_pt(file):
  return torch.load(file).cuda()

def ema(base, next, step):
    scale_ma = 1. / (step + 1)
    return next * scale_ma + base * (1.- scale_ma)

def save_img(img, fname=None):
  img = np.array(img)[:,:,:]
  img = np.transpose(img, (1,2,0))  
  img = np.clip(img*255, 0, 255).astype(np.uint8)
  if fname is not None:
    imageio.imsave(fname, np.array(img))
    imageio.imsave('result.jpg', np.array(img))

def makevid(seq_dir, size=None):
  # out_sequence = seq_dir + '/%05d.jpg'
  out_video = seq_dir + '.mp4'
  # !ffmpeg -y -v quiet -i $out_sequence $out_video
  moviepy.editor.ImageSequenceClip(img_list(seq_dir), fps=25).write_videofile(out_video, verbose=False) # , ffmpeg_params=ffmpeg_params, logger=None
  data_url = "data:video/mp4;base64," + b64encode(open(out_video,'rb').read()).decode()
  wh = '' if size is None else 'width=%d height=%d' % (size, size)
  return """<video %s controls><source src="%s" type="video/mp4"></video>""" % (wh, data_url)

!nvidia-smi -L
print('\nDone!')

GPU 0: Tesla T4 (UUID: GPU-7a836acb-d67c-7a78-86e5-bef2f5eedd81)

Done!


In [1]:
multilang = False #param {type:"boolean"}
translate = False #param {type:"boolean"}
#uploaded = files.upload()

In [2]:
import pandas as pd
from google.colab import files 
import io
!rm dreams.xlsx
uploaded = files.upload()
todolist = pd.read_excel("dreams2.xlsx")

rm: cannot remove 'dreams.xlsx': No such file or directory


Saving dreams2.xlsx to dreams2.xlsx


In [3]:
for k in range(len(todolist)) :
  try : 
    sideX = 1280
    sideY = 720 
    duration =  todolist["DURATION"][k]

    #markdown > Config
    model = todolist["MODEL"][k] #'ViT-B/32' #param ['ViT-B/32', 'RN101', 'RN50x4', 'RN50']
    overscan = False
    keep = todolist["KEEP"][k]
    noise_scale = todolist["NOISE"][k]
    contrast = todolist["CONTRAST"][k]
    #markdown > Training
    steps = todolist["STEPS"][k] 
    samples = todolist["SAMPLES"][k]
    learning_rate = todolist["LEARNING RATE"][k]
    show_freq = todolist["FREQUENCY"][k]
    #markdown > Tricks
    diverse = todolist["DIVERSE"][k]
    expand = todolist["EXPAND"][k] 
    progressive_grow = todolist["PROGRESSIVE GROW"][k]
    fps = 25
    if multilang: model = 'ViT-B/32' # sbert model is trained with ViT

    model_clip, _ = clip.load(model)
    modsize = 288 if model == 'RN50x4' else 224
    xmem = {'RN50':0.5, 'RN50x4':0.16, 'RN101':0.33}
    if 'RN' in model:
      samples = int(samples * xmem[model])

    if diverse != 0:
      samples = int(samples * 0.5)
            
    norm_in = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

    text_file = todolist["TEST"][k]

    texts = texts = todolist["TEXT"][k].split('\n')
    texts = [tt.strip() for tt in texts if len(tt.strip())>0 and tt[0] != '#']
    print(' text file:', text_file)

    workdir = os.path.join(work_dir, basename(text_file))
    workdir += '-%s' % model if 'RN' in model.upper() else ''
    !rm -rf $workdir
    os.makedirs(workdir, exist_ok=True)

    outpic = ipy.Output()
    outpic
      
    # make init
    global params_start, params_ema
    params_shape = [1, 3, sideY, sideX//2+1, 2]
    params_start = torch.randn(*params_shape).cuda() # random init
    params_ema = 0.

    if resume is True:
      # print(' resuming from', resumed)
      params, _ = fft_image([1, 3, sideY, sideX], resume = params_pt, sd=1.)
      if keep > 0:
        params_ema = params[0].detach()
      torch.save(params_pt, os.path.join(workdir, '000-start.pt'))
    else:
      torch.save(params_start, os.path.join(workdir, '000-start.pt'))

    torch.save(params_start, 'init.pt') # final init

    prev_enc = 0
    def process(txt, num):

      global params_start
      sd = 0.01
      if keep > 0: sd = keep + (1-keep) * sd
      params, image_f = fft_image([1, 3, sideY, sideX], resume='init.pt', sd=sd)
      image_f = to_valid_rgb(image_f)
      
      if progressive_grow is True:
        lr1 = learning_rate * 2
        lr0 = lr1 * 0.01
      else:
        lr0 = learning_rate
      optimizer = torch.optim.Adam(params, lr0)
        
      print(' ref text: ', txt)
      if translate:
        translator = Translator()
        txt = translator.translate(txt, dest='en').text
        print(' translated to:', txt)
      if multilang:
          model_lang = SentenceTransformer('clip-ViT-B-32-multilingual-v1').cuda()
          txt_enc = model_lang.encode([txt], convert_to_tensor=True, show_progress_bar=False).detach().clone()
          del model_lang
      else:
          txt_enc = model_clip.encode_text(clip.tokenize(txt).cuda()).detach().clone()

      out_name = '%03d-%s' % (num+1, txt.translate(str.maketrans(dict.fromkeys(list("\n',—|!?/:;\\"), ""))).replace(' ', '_').replace('"', ''))
      tempdir = os.path.join(workdir, out_name)
      !rm -rf $tempdir
      os.makedirs(tempdir, exist_ok=True)

      pbar = ProgressBar(steps) #  // save_freq
      for i in range(steps):
        loss = 0
        
        noise = noise_scale * torch.randn(1, 1, *params[0].shape[2:4], 1).cuda() if noise_scale > 0 else 0.
        img_out = image_f(noise)

        imgs_sliced = slice_imgs([img_out], samples, modsize, norm_in, overscan=overscan, micro=None)
        out_enc = model_clip.encode_image(imgs_sliced[-1])
        loss -= torch.cosine_similarity(txt_enc, out_enc, dim=-1).mean()
        if diverse != 0:
          imgs_sliced = slice_imgs([image_f(noise)], samples, modsize, norm_in, overscan=overscan, micro=None)
          out_enc2 = model_clip.encode_image(imgs_sliced[-1])
          loss += diverse * torch.cosine_similarity(out_enc, out_enc2, dim=-1).mean()
          del out_enc2; torch.cuda.empty_cache()
        if expand > 0:
          global prev_enc
          if i > 0:
            loss += expand * torch.cosine_similarity(out_enc, prev_enc, dim=-1).mean()
          prev_enc = out_enc.detach()
        del img_out, imgs_sliced, out_enc; torch.cuda.empty_cache()

        if progressive_grow is True:
          lr_cur = lr0 + (i / steps) * (lr1 - lr0)
          for g in optimizer.param_groups: 
            g['lr'] = lr_cur

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % show_freq == 0:
          with torch.no_grad():
            img = image_f(contrast=contrast).cpu().numpy()[0]
          save_img(img, os.path.join(tempdir, '%05d.jpg' % (i // show_freq)))
          outpic.clear_output()
          with outpic:
            display(Image('result.jpg'))
          del img

        pbar.upd()

      if keep > 0:
        global params_start, params_ema
        params_ema = ema(params_ema, params[0].detach(), num+1)
        torch.save((1-keep) * params_start + keep * params_ema, 'init.pt')

      torch.save(params[0], '%s.pt' % os.path.join(workdir, out_name))

    for i, txt in enumerate(texts):
        process(txt, i)

    vsteps = int(duration * fps / len(texts))
    tempdir = os.path.join(workdir, '_final')
    !rm -rf $tempdir
    os.makedirs(tempdir, exist_ok=True)
  except : print ('Failure : ' + todolist["TEST"][k])


Failure : monkey_1
Failure : monkey_2
Failure : monkey_3
Failure : monkey_4
Failure : monkey_5
Failure : monkey_6
Failure : monkey_7
Failure : monkey_8
Failure : monkey_9
Failure : monkey_10
Failure : monkey_11
Failure : monkey_12
Failure : monkey_13
Failure : monkey_14
Failure : monkey_15
Failure : monkey_16
Failure : monkey_17
Failure : monkey_18
Failure : monkey_19
Failure : view_20
Failure : view_21
Failure : view_22
Failure : view_23
Failure : view_24
Failure : view_25
Failure : view_26
Failure : view_27
Failure : view_28
Failure : view_29
Failure : view_30
Failure : view_31
Failure : view_32
Failure : view_33
Failure : view_34
Failure : view_35
Failure : view_36
Failure : view_37
Failure : view_38
Failure : view_39
Failure : view_40
Failure : view_41
Failure : view_42
Failure : view_43
Failure : view_44
Failure : view_45
Failure : view_46
Failure : view_47
Failure : view_48
Failure : view_49
Failure : view_50
Failure : view_51
Failure : view_52
Failure : view_53
Failure : view_54

In [83]:
print(workdir)

/G/MyDrive/illustra/monkey_1


In [ ]:
!rm workdir

In [89]:
images = glob.glob(workdir + '/')
for image in images : 
  print(image)
  files.download(image)

/G/MyDrive/illustra/monkey_1/


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>